In [ ]:
pip install python-chess

In [ ]:
import pandas as pd
import numpy as np
import chess.pgn
import chess
from random import randint
from random import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, losses

In [ ]:
#pos2vec = tf.keras.models.load_model("")

In [ ]:
def fen2bitstring(fen):
    board = chess.Board(fen)
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard
    

In [ ]:
def randPosiGeneratorFromGame(game):
    random_positions_array = []
    positions = []
    board = game.board()

    for move in game.mainline_moves():
        if not board.is_capture(move):
            position = board.fen()
            bitstring_position = fen2bitstring(position)
            positions.append(bitstring_position)

        board.push(move)

        no_capture_posis = len(positions)
    indices = list(range(5, no_capture_posis))
    shuffle(indices)
    selected_indices = indices[:10]

    random_positions_array = [positions[index] for index in selected_indices]

    return random_positions_array

In [ ]:
pgn_file = open("../input/ccrl-dataset/CCRLdb.pgn")

random_positions_white_win_bitstring = []
random_positions_black_win_bitstring = []

i = 0
while i < (200000):
    if i > 0:
        game = chess.pgn.read_game(pgn_file)
        if game.headers["Result"] == "1-0":
            temp1 = randPosiGeneratorFromGame(game)
            random_positions_white_win_bitstring += temp1
        elif game.headers["Result"] == "0-1":
            temp2 = randPosiGeneratorFromGame(game)
            random_positions_black_win_bitstring += temp2
        if game is None:
            break
    i = i + 1
        
pgn_file.close()

In [ ]:
len(random_positions_white_win_bitstring)

In [ ]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring, dtype=tf.float32)
black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring, dtype=tf.float32)

In [ ]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

In [ ]:
concatenated_dataset = white_win_dataset.concatenate(black_win_dataset)

In [ ]:
num_samples = len(concatenated_dataset)
train_dataset = concatenated_dataset.shuffle(num_samples)
print('Train Dataset Size:', len(train_dataset))

In [ ]:
def scheduler(epoch, lr):
    return lr * 0.98

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler)

# Set the initial learning rate
initial_learning_rate = 0.005

In [ ]:
train_data = np.array(list(train_dataset.as_numpy_iterator()))

In [ ]:
def train_autoencoders():  #[[e1, d1], [c1], [e2, d2], [c2], [e3, d3], [c3], [e4, d4], [o]]
    pos2vec = tf.keras.Sequential([
    layers.Dense(600, activation='relu', input_shape=(773,)),
    layers.Dense(400, activation='relu', input_shape=(600,)),
    layers.Dense(200, activation='relu', input_shape=(400,)),
    layers.Dense(100, activation='relu', input_shape=(200,)),
    layers.Dense(200, activation='relu', input_shape=(100,)),
    layers.Dense(400, activation='relu', input_shape=(200,)),
    layers.Dense(600, activation='relu', input_shape=(400,)),
    layers.Dense(773, activation='relu', input_shape=(600,)),
    ])
    
    return pos2vec

In [ ]:
pos2vec = train_autoencoders()
pos2vec.compile(optimizer = 'adam', loss = losses.MeanSquaredLogarithmicError())
pos2vec.fit(train_data, train_data, epochs = 200, batch_size = 1000, callbacks=[lr_scheduler])

In [ ]:
weights = []
weights.append(pos2vec.layers[0].get_weights())
weights.append(pos2vec.layers[1].get_weights())
weights.append(pos2vec.layers[2].get_weights())
weights.append(pos2vec.layers[3].get_weights())
    
pos2vec.save("pos2vec_final_draft1_200000.h5")

In [ ]:
trained_pos2vec_weights = weights

In [ ]:
np.save("pos2vec_final_draft1_200000", trained_pos2vec_weights)